In [ ]:
#| default_exp estimate

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

import numpy as np
from fastcore.test import test_close, test_eq
from typing import Tuple, List

from matching_separable_simuls.utils import legendre_polynomials, quantile_transform



In [ ]:
#| export

def generate_bases(
    nx: np.ndarray,                              # the numbers of men of each type
    my: np.ndarray,                              # the numbers of women of each type
    degrees: List[Tuple[int, int]],              # the list of degrees for polynomials in `x` and `y`
    ) -> Tuple[np.ndarray, List[str]]:           # the matrix of base functions and their names
    """ generates the bases for a semilinear specification """
    n_bases = 3 + len(degrees)
    n_types_men, n_types_women = nx.size, my.size
    types_men, types_women = np.arange(n_types_men), np.arange(n_types_women)
    base_funs = np.zeros((n_types_men, n_types_women, n_bases))
    base_names = [None]*n_bases
    base_funs[:, :, 0] = 1.0
    base_names[0] = "1"
    for y in types_women:
        base_funs[:, y, 1] = np.where(types_men > y, 1.0, 0.0)
        base_funs[:, y, 2] = np.where(types_men > y, types_men-y, 0)
    base_funs[:, :, 2] /= (n_types_men + n_types_women)/2
    base_names[1] = "1(x>y)"
    base_names[2] = "max(x-y,0)"
    # we quantile-transform nx and my
    q_nx = quantile_transform(nx)
    q_my = quantile_transform(my)
    # and we use the Legendre polynomials on [0,1]
    max_deg_x = max(degree[0] for degree in degrees)
    max_deg_y = max(degree[1] for degree in degrees)
    polys_x = legendre_polynomials(q_nx, max_deg_x, a=0)
    polys_y = legendre_polynomials(q_my, max_deg_y, a=0)
    i_base = 3
    for deg_x, deg_y in degrees:
        poly_x = polys_x[:, deg_x]
        poly_y = polys_y[:, deg_y]
        base_funs[:, :, i_base] = np.outer(poly_x, poly_y)
        base_names[i_base] = f"x^{deg_x} * y^{deg_y}"
        i_base += 1
    return base_funs, base_names


In [ ]:
#| hide
nx = np.array([1.0, 2.0, 4.0])
my = np.array([3.0, 1.0])
bases, base_names = generate_bases(nx, my, [(1,1), (1,2), (2,1)])
test_close(bases[:, :, 0], 1.0)
print(base_names)

['1', '1(x>y)', 'max(x-y,0)', 'x^1 * y^1', 'x^1 * y^2', 'x^2 * y^1']


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()